In [2]:
!pip install pymdptoolbox

  Preparing metadata (setup.py) ... done
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25656 sha256=17fe6686641cc58ee322276f984f9d4cdf3fdfb7899dcbc9335adc0032a50a8f
  Stored in directory: /root/.cache/pip/wheels/2b/e7/c7/d7abf9e309f3573a934fed2750c70bd75d9e9d901f7f16e183
Successfully built pymdptoolbox


# Mining Environment

In [3]:
import mdptoolbox, mdptoolbox.example
import numpy as np
import enum

In [4]:
class State:
    def __init__(self, l_a, l_h, b_e, match="relevant"):
        self.length_a = l_a
        self.length_h = l_h
        self.blocks_e = b_e
        self.match = match
        self.match_dict = {'irrelevant': 0, 'relevant': 1, 'active': 2}


    def __hash__(self):
        return hash((self.length_a, self.length_h, self.blocks_e, self.match))

    def __eq__(self, other):
        try:
            return (self.length_a, self.length_h, self.blocks_e, self.match) == (other.length_a, other.length_h, other.blocks_e, other.match)
        except:
            return False

    def __ne__(self, other):
        return not (self == other)

    def __repr__(self):
        return "(%d, %d, %d, %s)" % (self.length_a, self.length_h, self.blocks_e, self.match)

    def to_numpy(self):
      return np.array([self.length_a, self.length_h, self.blocks_e, self.match_dict[self.match]])


In [5]:
class MiningEnv():

  def __init__(self, p, stale, gamma, cost, cutoff=20, lam=0):
    self.p = p
    self.cutoff = cutoff
    self.lam = lam
    self.match_cases = ["irrelevant", "relevant", "active"]
    self.a_cost = cost * p
    self.h_cost = cost * (1 - p)
    self.q = 1 - p - lam
    self.stale = stale
    self.gamma = gamma
    self.current_state = State(0, 0, 0, 'irrelevant')
    state_count = 0
    self.states = {}
    self.states_inverted = {}
    for l_a in range(self.cutoff + 1):
      for l_h in range(self.cutoff + 1):
        for b_e in range(l_a + 1):
          if self.lam == 0 and b_e > 0:
            continue
          for match in self.match_cases:
            state = State(l_a, l_h, b_e, match)
            self.states[state_count] = state
            self.states_inverted[state] = state_count
            state_count += 1
    self.states_counter = state_count

    # transition matrices
    P_adopt = np.zeros(shape=(state_count, state_count))
    P_override = np.zeros(shape=(state_count, state_count))
    P_match = np.zeros(shape=(state_count, state_count))
    P_wait = np.zeros(shape=(state_count, state_count))

    # reward matrices
    R_adopt = np.empty(shape=(state_count, state_count), dtype=object)
    R_override = np.empty(shape=(state_count, state_count), dtype=object)
    R_match = np.empty(shape=(state_count, state_count), dtype=object)
    R_wait = np.empty(shape=(state_count, state_count), dtype=object)

    for state_idx, state in self.states.items():
      l_a = state.length_a
      l_h = state.length_h
      b_e = state.blocks_e
      match = state.match

      # adopt action transition matrix
      # attacker mines next block
      P_adopt[state_idx, self.states_inverted[State(1, 0, 0, "irrelevant")]] = self.p
      R_adopt[state_idx, self.states_inverted[State(1, 0, 0, "irrelevant")]] = (-self.a_cost, l_h - self.h_cost)

      # eclipsed node mines next block
      if self.lam != 0:
        P_adopt[state_idx, self.states_inverted[State(1, 0, 1, "irrelevant")]] = self.lam
        R_adopt[state_idx, self.states_inverted[State(1, 0, 1, "irrelevant")]] = (-self.a_cost, l_h - self.h_cost)

      # honest network mines next block
      P_adopt[state_idx, self.states_inverted[State(0, 1, 0, "relevant")]] = self.q*(1-self.stale)
      R_adopt[state_idx, self.states_inverted[State(0, 1, 0, "relevant")]] = (-self.a_cost, l_h - self.h_cost)

      # network mines state block
      P_adopt[state_idx, self.states_inverted[State(0, 0, 0, "irrelevant")]] = self.q * self.stale
      R_adopt[state_idx, self.states_inverted[State(0, 0, 0, "irrelevant")]] = (-self.a_cost, l_h - self.h_cost)

      # override action transition matrix
      if l_a > l_h:
        payout = (l_h+1)*(l_a - b_e)//l_a
        new_b_e = b_e - (l_h+1 - payout)
      # attacker mines next block
        P_override[state_idx, self.states_inverted[State(l_a - l_h, 0, new_b_e, "irrelevant")]] = self.p
        R_override[state_idx, self.states_inverted[State(l_a - l_h, 0, new_b_e, "irrelevant")]] = (payout - self.a_cost, b_e - new_b_e - self.h_cost)

      # eclipsed node mines next block
        if self.lam != 0:
          P_override[state_idx, self.states_inverted[State(l_a - l_h, 0, new_b_e + 1, "irrelevant")]] = self.lam
          R_override[state_idx, self.states_inverted[State(l_a - l_h, 0, new_b_e + 1, "irrelevant")]] = (payout - self.a_cost, b_e - new_b_e - self.h_cost)

      # network mines next block
        P_override[state_idx, self.states_inverted[State(l_a-l_h-1, 1, new_b_e, "relevant")]] = self.q*(1 - self.stale)
        R_override[state_idx, self.states_inverted[State(l_a-l_h-1, 1, new_b_e, "relevant")]] = (payout - self.a_cost, b_e - new_b_e - self.h_cost)

      # network mines stale block
        P_override[state_idx, self.states_inverted[State(l_a-l_h-1, 0, new_b_e, "irrelevant")]] = self.q*self.stale
        R_override[state_idx, self.states_inverted[State(l_a-l_h-1, 0, new_b_e, "irrelevant")]] = (payout - self.a_cost, b_e - new_b_e - self.h_cost)

      else:
        P_override[state_idx, state_idx] = 1
        R_override[state_idx, state_idx] = (-100, -100)

      # perform adopt and override after cutoff reached
      if l_a == self.cutoff or l_h == self.cutoff:
        P_match[state_idx, state_idx] = 1
        R_match[state_idx, state_idx] = (-100, -100)
        P_wait[state_idx, state_idx] = 1
        R_wait[state_idx, state_idx] = (-100, -100)
        continue


      # match action transition matrix
      if match == 'relevant' and l_a >= l_h and l_h > 0:
        payout = (l_h)*(l_a - b_e)//l_a
        new_b_e = b_e - (l_h - payout)

        # attacker mines next block
        P_match[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e, "active")]] = self.p
        R_match[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e, "active")]] =  (-self.a_cost, -self.h_cost)

        # eclipsed node mines next block
        if self.lam != 0:
          P_match[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e+1, "active")]] = self.lam
          R_match[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e+1, "active")]] = (-self.a_cost, -self.h_cost)

        # network mines next block after pool's head
        P_match[state_idx, self.states_inverted[State(l_a - l_h, 1, new_b_e, "relevant")]] = self.gamma * self.q * (1 - self.stale)
        R_match[state_idx, self.states_inverted[State(l_a - l_h, 1, new_b_e, "relevant")]] = (payout - self.a_cost, b_e - new_b_e - self.h_cost)

        # network mines next block after other's head
        P_match[state_idx, self.states_inverted[State(l_a, l_h + 1, b_e, "relevant")]] = (1-self.gamma) * self.q * (1 - self.stale)
        R_match[state_idx, self.states_inverted[State(l_a, l_h + 1, b_e, "relevant")]] = (-self.a_cost, -self.h_cost)

        # network mines stale block
        P_match[state_idx, self.states_inverted[State(l_a, l_h, b_e, "active")]] = self.q * self.stale
        R_match[state_idx, self.states_inverted[State(l_a, l_h, b_e, "active")]] = (-self.a_cost, -self.h_cost)
      else:
        P_match[state_idx, state_idx] = 1
        R_match[state_idx, state_idx] = (-100, -100)

      # wait action transition matrix
      if match == 'active' and l_a >= l_h and l_h > 0:
        payout = (l_h)*(l_a - b_e)//l_a
        new_b_e = b_e - (l_h - payout)

        # attacker mines next block
        P_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e, "active")]] = self.p
        R_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e, "active")]] = (-self.a_cost, -self.h_cost)

        # eclipsed node mines next block
        if self.lam != 0:
          P_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e+1, "active")]] = self.lam
          R_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e+1, "active")]] = (-self.a_cost, -self.h_cost)

        # network mines after the pool's head
        P_wait[state_idx, self.states_inverted[State(l_a - l_h, 1, new_b_e, "relevant")]] = self.gamma * self.q * (1 - self.stale)
        R_wait[state_idx, self.states_inverted[State(l_a - l_h, 1, new_b_e, "relevant")]] = (payout - self.a_cost, b_e - new_b_e - self.h_cost)

        # network mines after other's head
        P_wait[state_idx, self.states_inverted[State(l_a, l_h + 1, b_e, "relevant")]] = (1-self.gamma) * self.q * (1 - self.stale)
        R_wait[state_idx, self.states_inverted[State(l_a, l_h + 1, b_e, "relevant")]] = (-self.a_cost, -self.h_cost)

        # network mines stale block
        P_wait[state_idx, self.states_inverted[State(l_a, l_h, b_e, "active")]] = self.q * self.stale
        R_wait[state_idx, self.states_inverted[State(l_a, l_h, b_e, "active")]] = (-self.a_cost, -self.h_cost)

      else:
        # attacker mines next block
        P_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e, "irrelevant")]] = self.p
        R_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e, "irrelevant")]] = (-self.a_cost, -self.h_cost)

        # eclipsed node mines next block
        if self.lam != 0:
          P_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e+1, "irrelevant")]] = self.lam
          R_wait[state_idx, self.states_inverted[State(l_a + 1, l_h, b_e+1, "irrelevant")]] = (-self.a_cost, -self.h_cost)

        # network mines next block
        P_wait[state_idx, self.states_inverted[State(l_a, l_h + 1, b_e, "relevant")]] = self.q * (1 - self.stale)
        R_wait[state_idx, self.states_inverted[State(l_a, l_h + 1, b_e, "relevant")]] = (-self.a_cost, -self.h_cost)

        # network mines stale block
        P_wait[state_idx, self.states_inverted[State(l_a, l_h, b_e, "irrelevant")]] = self.q * self.stale
        R_wait[state_idx, self.states_inverted[State(l_a, l_h, b_e, "irrelevant")]] = (-self.a_cost, -self.h_cost)

    self.P = np.array([P_wait, P_adopt, P_override, P_match])
    self.R = np.array([R_wait, R_adopt, R_override, R_match])

    # state dimension
    self.S = self.P.shape[1]

    # action dimension
    self.A = len(self.P)

  def reset(self):
    self.current_state = State(0, 0, 0, "irrelevant")
    return self.current_state.to_numpy()

  def step(self, action):
    p_s_new = np.random.random()
    p = 0
    s_new = -1
    while (p < p_s_new) and (s_new < (self.S - 1)):
      s_new = s_new + 1
      p = p + self.P[action][self.states_inverted[self.current_state], s_new]
    try:
      r = self.R[action][self.states_inverted[self.current_state], s_new]
    except IndexError:
      try:
        r = self.R[self.states_inverted[self.current_state], action]
      except IndexError:
        r = self.R[self.states_inverted[self.current_state]]
    self.current_state = self.states[s_new]
    return (self.current_state.to_numpy(), np.array(r).astype(np.float32))



In [6]:
p = 0.4
stale = 0.1
gamma = 0.5
cost = 0
env = MiningEnv(p=p, stale=stale, gamma = gamma, cost = cost)

In [7]:
env.step(0)

(array([1, 0, 0, 0]), array([-0., -0.], dtype=float32))

# DQN

In [8]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

In [9]:
class Match(enum.IntEnum):
  irrelevant = 0,
  relevant = 1,
  active = 2

class Action(enum.IntEnum):
  wait = 0,
  adopt = 1,
  override = 2,
  match = 3

action_size = 4
state_size = 4

In [10]:
# Creating the architecture of the network
class Network(nn.Module):
  # number of input neurons = size of the dimensions of the state (8)
  # number of actions
  # random seed
  def __init__(self, state_size, action_size, seed = 42):
    super(Network, self).__init__()
    # Sets the seed for generating random numbers
    self.seed = torch.manual_seed(seed)
    # first full connection between the input layer and hidden layer
    self.fc1 = nn.Linear(state_size, 64)
    # second full connection layer between first hidden layer and second layer
    self.fc2 = nn.Linear(64, 128)
    # connetion between the second hidden layer and the output layer
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, action_size)

  # forward propogation from input layer to the output layer
  def forward(self, state):
    # applying activation function (propogate the signal from input layer
    # to the first hidden layer applying activation function)
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    return self.fc4(x)

## Training the agent

In [11]:
p = 0.4
stale = 0.1
gamma = 0.5
cost = 0
# setup up the environment
env = MiningEnv(p=p, stale=stale, gamma = gamma, cost = cost)

## Initialize hyper parameters


In [12]:
learning_rate = 5e-4
# size of each batch where the model will be trained
minibatch_size = 100
discount_factor = 0.99
# size of the replay buffer
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

## Implementing experience replay buffer

In [13]:
# Implementing experience replay
class ReplayMemory(object):
  # capacity -> size of the buffer
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  # append a new transition in the memory and ensures that memory contain
  # only 100000 transitions
  def push(self, event):
    self.memory.append(event)
    # check if buffer does not exceed the capacity
    if len(self.memory) > self.capacity:
      del self.memory[0]

  # batch_size -> number of experiences sampled in a batch
  # each experience contains (state,action,reward,nextstate,boolean done)
  def sample(self, batch_size):
    experiences = random.sample(self.memory, k=batch_size)
    # all the states corresponding to all experiences
    # next converting to torch tensor
    # finally move to the designated computing device
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    # actions
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    # rewards
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    # next states
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    return states, next_states, actions, rewards

## Implementing DQN class



In [14]:
class Agent():
  def __init__(self, state_size, action_size):
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.state_size = state_size
    self.action_size = action_size

    # local q network of adversary
    self.a_local_qnetwork = Network(state_size, action_size).to(self.device)
    # target q network of adversary
    self.a_target_qnetwork = Network(state_size, action_size).to(self.device)
    # local q network of honest
    self.h_local_qnetwork = Network(state_size, action_size).to(self.device)
    # target q network of honest
    self.h_target_qnetwork = Network(state_size, action_size).to(self.device)

    # parameters are the weights of the network
    self.a_optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
    self.h_optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)

    # replay memory
    self.memory = ReplayMemory(replay_buffer_size)
    # timestep to decide when to learn from the experirences
    self.t_step = 0

# method to store exp and decide when to learn from them
  def step(self, state, action, reward, next_state):
    self.memory.push((state, action, reward, next_state))
    # when timestep reaches 4 the model will learn by taking a minibatch from the
    # replay buffer
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      # check if there are at least 100 exp in the buffer
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    # adding an extra dimension corresponding to the batch (it indicates to which batch this state belogns to)
    # note that always the batch index should be at the beginning
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    # set the local network to evaluation mode before forward pass
    # because as this is the forward pass we are making predictions
    self.a_local_qnetwork.eval()
    self.h_local_qnetwork.eval()
    # since we are in forward there is no need to calculate gradients
    with torch.no_grad():
      # predicting q values (forward pass)
      a_action_values = self.local_a_qnetwork(state)
      h_action_values = self.local_h_qnetwork(state)
    # resetting model to traning mode
    self.a_local_qnetwork.train()
    self.h_local_qnetwork.train()
    # select an action based on epsilon greedy policy
    # we generate a random number R and if R > epsilon ? we choose the maximum predicted q value
    # : select a random aciton
    if random.random() > epsilon:
      # argmax returns the index of maxium value of a numpy array
      # .data selects the tensor data and .numpy() is used to convert tensor to numpy array
      a_actions = a_action_values.cpu().data.squeeze(0)
      h_actions = h_action_values.cpu().data.squeeze(0)
      return torch.argmax(a_actions / (a_actions + h_actions)).item()
    else:
      # selects a random index 0,1,2,3
      return random.choice(np.arange(self.action_size))

  # allows agent to learn based on the minibatch
  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    # to compute the target q value we need the maxium q value for the next state
    # use the target network to get the q values for all the actions from that next state
    # next_q_targets = self.a_target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    # (100, 4)
    next_a_q_targets = self.a_target_qnetwork(next_states).detach()
    next_h_q_targets = self.h_target_qnetwork(next_states).detach()
    # (100, 1)
    a_ = torch.stack([next_a_q_targets[idx] / (next_a_q_targets[idx] + next_h_q_targets[idx]) for idx, _ in enumerate(next_a_q_targets)]).max(1)[1].unsqueeze(1)
    # (100, 1)
    q_a_targets = rewards[:, 0].unsqueeze(1) + discount_factor * next_a_q_targets.gather(1, a_)
    q_h_targets = rewards[:, 1].unsqueeze(1) + discount_factor * next_h_q_targets.gather(1, a_)

    # q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    # forward propogate the states to get the predicted q values
    q_a_expected = self.a_local_qnetwork(states).gather(1, actions)
    q_h_expected = self.h_local_qnetwork(states).gather(1, actions)

    # loss (mean squared error)
    loss_a = F.mse_loss(q_a_expected, q_a_targets)
    loss_h = F.mse_loss(q_h_expected, q_h_targets)

    # backpropogating the error to update the weights
    self.a_optimizer.zero_grad()
    self.h_optimizer.zero_grad()

    loss_a.backward()
    loss_h.backward()
    # single optimization step for updating the weights
    self.a_optimizer.step()
    self.h_optimizer.step()

    # updating the target network weights
    self.soft_update(self.a_local_qnetwork, self.a_target_qnetwork, interpolation_parameter)
    self.soft_update(self.h_local_qnetwork, self.h_target_qnetwork, interpolation_parameter)




In [63]:
a = torch.from_numpy(np.vstack([(1,2)])).float()
b = torch.from_numpy(np.vstack([(5,3)])).float()

In [47]:
c = torch.stack([a[idx] / (a[idx] + b[idx]) for idx, _ in enumerate(a)]).max(1)[1]
c

tensor([1])

In [66]:
a_ = a.cpu().data.squeeze(0)
b_ = b.cpu().data.squeeze(0)

In [67]:
a_, b_

(tensor([1., 2.]), tensor([5., 3.]))

In [71]:
torch.argmax(a_ / (a_ + b_))

tensor(1)

In [42]:
r = a[:, 0].unsqueeze(1)

In [44]:
action_values = torch.tensor([2.5, 1.2, 10.1, 1.4])
np.argmax(action_values.cpu().data.numpy())

2